## Libraries

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')
from neo4j import GraphDatabase

## Connect to Neo4J

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "Croma2023"))
session = driver.session()

## Create entities and relate them to events

In [3]:
query = f'''MATCH (e:Event) UNWIND e.KitID AS id_val
WITH DISTINCT id_val
MERGE (:Entity {{ID:id_val, EntityType:"Kit"}})'''
session.run(query).single()

In [4]:
query = f'''MATCH (e:Event) UNWIND e.KitID AS kitID WITH e,kitID
MATCH (n:Entity {{EntityType: "Kit"}}) WHERE kitID = n.ID
MERGE (e)-[:CORR]->(n)'''
session.run(query).single()

## Create directly follows relationships between events related to the same entity

In [5]:
query = f'''MATCH (n:Entity {{EntityType:"Kit"}})
MATCH (n)<-[:CORR]-(e)
WITH n, e AS nodes ORDER BY e.timestamp, ID(e)
WITH n, collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH n, event_node_list[i] AS e1, event_node_list[i+1] AS e2
MERGE (e1)-[df:DF {{EntityType:n.EntityType, ID:n.ID, Duration: duration.between(e1.timestamp , e2.timestamp).seconds}}]->(e2)
'''
session.run(query)

## Create sterilization cycles as runs for each entity

In [6]:
#add start label to Entrada Material Sucio events
query = f'''MATCH (e:Event)
WHERE e.Activity =  "Entrada Material Sucio"
OPTIONAL MATCH (f:Event) - [:DF] -> (e)
SET e:NewKitEvent, f:LastKitEvent
 '''
session.run(query).single()

In [7]:
#add end label to commisionada events
query = f'''MATCH (e:Event)
WHERE e.Activity =  "Comisionado"
OPTIONAL MATCH (f:Event) <- [:DF] - (e)
SET e:LastKitEvent, f:NewKitEvent
'''
session.run(query).single()

In [8]:
#add end label to commisionada events
query = f'''MATCH (e:Event)
WHERE not () - [:DF] -> (e)
SET e:NewKitEvent
'''
session.run(query).single()

In [9]:
#add end label to commisionada events
#query = f'''MATCH (e:Event:LastKitEvent) - [:DF] -> (f:Event)
#where f.Activity <> "Comisionado"
#set f:NewKitEvent
# '''
#session.run(query).single()

In [10]:
query = f'''MATCH (f:Event) - [:DF] -> (e:Event)
where duration.between(f.timestamp, e.timestamp).days > 3
SET e:NewKitEvent, f:LastKitEvent
 '''
session.run(query).single()

In [11]:
#Find nodes that are followed by Entrada directly
query = f'''MATCH (start_event:Event:NewKitEvent:LastKitEvent)
            MATCH (start_event) - [:CORR] -> (k:Entity)
            CALL apoc.refactor.cloneNodes([k])
            YIELD input, output as new_k, error
            SET new_k:Run
            remove new_k:Entity
            SET start_event:OnlyKitEvent
            remove start_event:NewKitEvent:LastKitEvent
            //CREATE (new_k) <- [:HAS_RUN] - (k)
            WITH start_event, k, new_k
            CALL {{WITH start_event, k, new_k
            MATCH (start_event) - [r:CORR] -> (k)
            DELETE r
            CREATE (start_event) - [:CORR] -> (new_k)}}
'''
session.run(query).single()

In [12]:
#Find paths from start to end and create Run nodes
query = f'''MATCH (start_event:Event:NewKitEvent)
CALL apoc.path.expandConfig(start_event, {{relationshipFilter: "DF>", 
labelFilter:"+Event|/LastKitEvent"}})
YIELD path
MATCH (start_event) - [:CORR] -> (k:Entity)
CALL apoc.refactor.cloneNodes([k])
YIELD input, output as new_k, error
SET new_k:Run
remove new_k:Entity
//CREATE (new_k) <- [:HAS_RUN] - (k)
WITH nodes(path) as events, k, new_k
            CALL {{WITH events, k, new_k
             UNWIND events as  kit_event
            MATCH (kit_event) - [r:CORR] -> (k)
            DELETE r
            CREATE (kit_event) - [:CORR] -> (new_k)}}
 '''
session.run(query).single()

In [13]:
query = f'''match (e:Entity)
set e:Run
remove e:Entity'''
session.run(query).single()

In [14]:
query = f'''match (r:Run) 
where not (r) <- [:CORR] - ()
detach delete r
'''
session.run(query).single()

In [15]:
query = f'''match (r:Run) UNWIND r.ID AS id_val
WITH DISTINCT id_val
MERGE (:Entity {{ID:id_val, EntityType:"Kit"}})
'''
session.run(query).single()

In [16]:
query = f'''MATCH (r:Run) 
UNWIND r.ID AS kitID 
WITH r, kitID
MATCH (n:Entity {{EntityType: "Kit"}}) WHERE kitID = n.ID
MERGE (r)<-[:HAS_RUN]-(n)'''
session.run(query).single()

In [17]:
query = f'''MATCH (startEvent:NewKitEvent)-[:CORR]->(c:Run) 
MATCH (endEvent:LastKitEvent)-[:CORR]->(c) 
Match path = (startEvent) - [:DF*] -> (endEvent)
set c.startTimestamp = startEvent.timestamp, 
c.endTimestamp=endEvent.timestamp, 
c.activities = startEvent.Activity
with tail(nodes(path)) as pathNodes, c
foreach (event in pathNodes | 
                set c.activities = c.activities + ", " + event.Activity)
'''
session.run(query).single()

In [ ]:
query = f'''match (r:Run) <- [:CORR] - (e:Event)
where not (e) - [:DF] -> () and r.startTimestamp is null
UNWIND r.ID AS kitID
WITH r, kitID, e
MATCH (startEvent:NewKitEvent)-[:CORR]->(r) 
Match path = (startEvent) - [:DF*] -> (e)
set r.startTimestamp = startEvent.timestamp, 
r.endTimestamp=e.timestamp, 
e:LastKitEvent,
r.activities = startEvent.Activity
with tail(nodes(path)) as pathNodes, r
foreach (event in pathNodes | 
                set r.activities = r.activities + ", " + event.Activity)'''
session.run(query).single()

In [18]:
query = f'''MATCH (startEvent:OnlyKitEvent)-[:CORR]->(c:Run) 
set c.startTimestamp = startEvent.timestamp, 
c.endTimestamp=startEvent.timestamp, 
c.activities = startEvent.Activity
'''
session.run(query).single()

In [ ]:
query = f'''match (r:Run) <- [:CORR] - (e:Event)
where r.startTimestamp is null 
unwind r.ID as kitID
with r, count(e) as c, e
where c = 1
set e:OnlyKitEvent, 
r.startTimestamp = e.timestamp, 
r.endTimestamp = e.timestamp, 
r.activities = e.Activity
remove e:NewKitEvent:LastKitEvent'''
session.run(query).single()

In [19]:
#query = f'''
#Match (e:Entity) 
#WITH  e AS cases ORDER BY e.startTimestamp, ID(e)
#WITH collect(cases) AS cases_list
#UNWIND range(0, size(cases_list)-1) AS i
#with cases_list[i] as node, i
#Match (ev:Event) - [:CORR] -> (node)
#set node.CaseID = node.ID + "-CN" + i,
#ev.CaseID = node.ID + "-CN" + i
#'''
#session.run(query).single()

In [20]:
query = f'''
Match (e:Run) 
WITH  e AS cases ORDER BY e.startTimestamp, ID(e)
WITH collect(cases) AS cases_list
UNWIND range(0, size(cases_list)-1) AS i
with cases_list[i] as node, i
Match (ev:Event) - [:CORR] -> (node)
set node.CaseID = node.ID + "-CN" + i,
ev.CaseID = node.ID + "-CN" + i
'''
session.run(query).single()

In [21]:
query = f'''match (r:Run)
return r.CaseID, r.startTimestamp, r.endTimestamp, r.startTimestamp.month
'''
frame = pd.DataFrame(session.run(query).data())
frame['r.startTimestamp'] = pd.to_datetime(frame['r.startTimestamp'], format='%Y-%m-%dT%H:%M:%S.%f%z')
frame['r.endTimestamp'] = pd.to_datetime(frame['r.endTimestamp'], format='%Y-%m-%dT%H:%M:%S.%f%z')
frame['durationDays'] = (frame['r.endTimestamp'] - frame['r.startTimestamp']).dt.total_seconds() / 60 / 60 / 24
frame['durationMinutes'] = (frame['r.endTimestamp'] - frame['r.startTimestamp']).dt.total_seconds() / 60 
for index, row in frame.iterrows():
    query = f'''match (r:Run {{CaseID: "{row['r.CaseID']}"}})
                set r.durationInMinutes = {row['durationMinutes']}
            '''
    session.run(query)

In [22]:
query = f'''match (e:Entity)-[:HAS_RUN]->(r:Run)
with count(r) as numberOfCycles, e
set e.numberOfCycles = numberOfCycles
return e.ID, numberOfCycles
'''
session.run(query).single()

<Record e.ID='HNB-OFT.002-10' numberOfCycles=24>

## Make Kit nodes

In [23]:
query = f'''match (e:Event)
UNWIND e.`Código` AS id
with distinct (id), count(distinct(e.NS)) as num
merge (k:Kit {{ID :id, numberOfUnits :num}})
return id, num'''
session.run(query).single()

<Record id='HNB-OFT.002' num=19>

In [24]:
query = f'''MATCH (e:Event)
MATCH (n:Entity)- [:HAS_RUN] -> (r:Run) 
where e.KitID = n.ID
set n.KitID = e.`Código`'''
session.run(query).single()

In [25]:
query = f'''MATCH (n:Entity) - [:HAS_RUN] -> (r:Run) 
Match (k:Kit)
where n.KitID = k.ID
merge (n) <- [:HAS_UNIT] - (k)
REMOVE n.KitID '''
session.run(query).single()

## Extract the process model

In [26]:
query = f'''MATCH ( e : Event ) WITH distinct e.Activity AS actName
MERGE ( c : Class {{ Name:actName, Type:"Activity", ID: actName}})'''
session.run(query).single()

In [27]:
query = f'''MATCH ( c : Class ) WHERE c.Type = "Activity"
MATCH ( e : Event ) WHERE c.Name = e.Activity
MERGE ( e ) -[:OBSERVED]-> ( c )'''
session.run(query).single()

In [28]:
query = f'''MATCH ( c : Class ) WHERE c.Type = "Activity"
MATCH ( e : Event ) WHERE c.Name = e.Activity
match (en:Entity) <-[:CORR]- (e)
MERGE ( en) -[:LOGGED_IN]-> ( c )'''
session.run(query).single()

In [29]:
query = f'''match (e:Entity)-[:HAS_RUN]->(r:Run)
MATCH ( c1 : Class ) <-[:OBSERVED]- ( e1 : Event ) -[df:DF]-> ( e2 : Event ) -[:OBSERVED]-> ( c2 : Class )
MATCH (e1) -[:CORR] -> (r) <-[:CORR]- (e2)
WHERE c1.Type = c2.Type AND r.EntityType = df.EntityType
WITH r.EntityType as EType,c1,count(df) AS df_freq,c2
MERGE ( c1 ) -[rel2:DF_C {{EntityType:EType}}]-> ( c2 ) ON CREATE SET rel2.count=df_freq'''
session.run(query).single()

## Add employees

In [30]:
query = f'''MATCH ( e : Event ) UNWIND e.Usuario AS employee 
WITH distinct employee
MERGE ( :Employee {{ Name:employee, Type:"Usuario", ID: employee}})'''
session.run(query).single()

In [31]:
query = f'''MATCH ( m : Employee ) WHERE m.Type = "Usuario"
MATCH ( e : Event ) WHERE m.Name = e.Usuario
CREATE ( e ) <-[:WORKED_ON]- ( m )'''
session.run(query).single()

In [32]:
query = f'''MATCH (emp:Employee)
MATCH (emp)-[:WORKED_ON]->(e)
WITH emp, e AS nodes ORDER BY e.timestamp, ID(e)
WITH emp, collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH emp, event_node_list[i] AS e1, event_node_list[i+1] AS e2
MERGE (e1)-[df:DF_EMP {{ID:emp.ID, startTimestamp: e1.timestamp, endTimestamp: e2.timestamp}}]->(e2)
'''
session.run(query).single()

## Add washing machines

In [33]:
query=f'''
MATCH (e:Event) UNWIND e.additionalInfo5 AS rack
WITH DISTINCT rack, SPLIT(rack, ' - ') AS result
with rack, result, apoc.text.regexGroups(result[0], '([0-9]+)')[0][0] AS rackNumber
MERGE (:Rack {{ID:rack, name: result[0], number:rackNumber, activity: result[1], EntityType:"Rack"}})
'''
session.run(query).single()

In [34]:
query=f'''
MATCH (e:Event) 
where e.Activity = "Carga L+D iniciada" or e.Activity = "Carga L+D liberada"
UNWIND e.additionalInfo1 AS wm
WITH DISTINCT wm, SPLIT(wm, ' - ') AS result, e
MERGE (:WashingMachine {{activity: rtrim(result[0]), machine: COALESCE(rtrim(result[1]), rtrim(e.additionalInfo2)), EntityType:"Washing Machine"}})
'''
session.run(query).single()

In [35]:
query = f'''
Match (r:Rack) 
Match (w:WashingMachine)
MATCH (e:Event) 
where e.Activity = "Carga L+D iniciada" or e.Activity = "Carga L+D liberada"
with SPLIT(e.additionalInfo1, ' - ') AS wm, e.additionalInfo5 as rack, r, w
where rack = r.ID and w.activity = wm[0]
merge (r) -[:RELATED_TO] -> (w)
'''
session.run(query).single()

In [36]:
query = f'''Match (w:WashingMachine)
MATCH (e:Event) 
where e.Activity = "Carga L+D iniciada" or e.Activity = "Carga L+D liberada"
with SPLIT(e.additionalInfo1, ' - ') AS wm, w, e
where w.activity = wm[0]
create (e) -[:WASHED_IN {{timestamp: e.timestamp}}] -> (w)
'''
session.run(query).single()

In [37]:
query = f'''Match (r:Rack)
MATCH (e:Event) 
where e.Activity = "Carga L+D iniciada" or e.Activity = "Carga L+D liberada"
with e.additionalInfo5 as rack, r, e
where rack = r.ID
create (e) -[:WASHED_ON {{timestamp: e.timestamp}}] -> (r)'''
session.run(query).single()

## Add sterilization machines

In [38]:
query=f'''MATCH (e:Event) 
where e.Activity = "Composición de cargas" or e.Activity = "Carga de esterilizador liberada"
UNWIND e.additionalInfo1 AS sm
WITH DISTINCT sm, SPLIT(sm, ' - ') AS result, e
MERGE (:SterilizationMachine {{activity: rtrim(result[0]), machine: rtrim(result[1]), EntityType:"Sterilization Machine",
 type: 
 case 
 when rtrim(toLower(result[1])) in ['amsco vpro', 'amsco eagle'] then 'Low Temperature Sterilization'
     else 'High Temperature Sterilization' end}})
     '''
session.run(query).single()

In [39]:
query = f'''Match (s:SterilizationMachine)
MATCH (e:Event) 
where e.Activity = "Composición de cargas" or e.Activity = "Carga de esterilizador liberada"
with SPLIT(e.additionalInfo1, ' - ') as sm, s, e
where rtrim(sm[1]) = s.machine
create (e) -[:STERILIZED_IN {{timestamp: e.timestamp}}] -> (s)
'''
session.run(query).single()

## Create directly follows for activities related to a case

In [40]:
query = f'''match ( e1 : Event ) -[df:DF]-> ( e2 : Event )
WHERE e1.CaseID = e2.CaseID
MERGE ( e1 ) -[rel:DF_CASE {{CaseID: e1.CaseID}}]-> ( e2 )'''
session.run(query).single()

## Create directly follows between cases

In [41]:
query = f'''match ( e1 : Event ) -[df:DF]-> ( e2 : Event )
WHERE e1.CaseID <> e2.CaseID
MERGE (e1) -[rel:DF_CYCLE {{PreviousCaseID: e1.CaseID, EndActivity: e1.Activity, endTimestamp: e1.timestamp,
 NextCaseID: e2.CaseID, StartActivity: e2.Activity, startTimestamp: e2.timestamp, 
  DurationBetweenSterilizations: Duration.between(e1.timestamp, e2.timestamp).minutes, KitID: e1.KitID}}]-> (e2)'''
session.run(query).single()

## Batch activities

In [42]:
#TODO check batches timestamp and the entry time for entrada
#TODO make separate batching relation for washing machines and sterilization machines
#TODO Question, should I batch activities with employee included or batch them as activity no matter the employee doing the job?

#query=f'''
#MATCH (e: Event)
#WITH e AS nodes ORDER BY e.timestamp, ID(e)
#WITH collect(nodes) AS event_node_list
#UNWIND range(0, size(event_node_list)-2) AS i
#WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
#WITH e1, e2, duration.between(e1.timestamp , e2.timestamp).minutes AS time_diff
#SET e1.batchID = CASE 
#    WHEN time_diff < 2 AND e1.Usuario = e2.Usuario AND e1.batchID IS NULL THEN "b" + ID(e1)
#    ELSE e1.batchID END,
#    e2.batchID = CASE 
#    WHEN time_diff < 2 AND e1.Usuario = e2.Usuario AND e1.batchID IS NOT NULL THEN e1.batchID
#    ELSE e2.batchID END
#'''
#session.run(query).single()

In [43]:
query=f'''MATCH (e: Event)
where e.Activity in ['Carga L+D iniciada', 'Carga L+D liberada', 'Composición de cargas', 'Carga de esterilizador liberada', 'Cargado en carro L+D']
WITH e AS nodes ORDER BY e.timestamp, ID(e)
WITH collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
WITH e1, e2, duration.between(e1.timestamp , e2.timestamp).minutes AS time_diff
SET e1.batchID = 
CASE 
    WHEN e1.batchID IS not NULL 
        THEN e1.batchID 
    ELSE 
        "b" + ID(e1)
            END,
    e2.batchID = 
CASE 
    WHEN time_diff < 3 AND (e1) - [:DF_EMP] -> (e2) 
        THEN e1.batchID
    WHEN time_diff >= 3 AND (e1) - [:DF_EMP] -> (e2) and e2.batchID is null
        THEN "b" + ID(e2)
    else 
        e2.batchID
            END'''
session.run(query)

In [44]:
query=f'''MATCH (e: Event)
where e.Activity in ['Carga L+D iniciada', 'Carga L+D liberada', 'Composición de cargas', 'Carga de esterilizador liberada', 'Cargado en carro L+D']
SET e.batchID = "b" + ID(e)
'''
session.run(query)

In [45]:
query=f'''MATCH (e: Event)
where e.Activity in ['Carga L+D iniciada', 'Carga L+D liberada', 'Composición de cargas', 'Carga de esterilizador liberada', 'Cargado en carro L+D']
WITH e AS nodes ORDER BY e.timestamp, ID(e)
WITH collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
SET e2.batchID = 
CASE 
    WHEN duration.between(e1.timestamp , e2.timestamp).minutes < 3 AND (e1) - [:DF_EMP] -> (e2) 
        THEN e1.batchID
    else 
        e2.batchID
            END'''
session.run(query)

In [46]:
query=f'''MATCH (e: Event)
where e.Activity in ['Comisionado', 'Producción montada', 'Entrada Material Sucio', 'Montaje']
SET e.batchID = "b" + ID(e)
'''
session.run(query)

In [47]:
query=f'''MATCH (e: Event)
where e.Activity in ['Comisionado', 'Producción montada', 'Entrada Material Sucio', 'Montaje']
WITH e AS nodes ORDER BY e.timestamp, ID(e)
WITH collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
SET e2.batchID = 
CASE 
    WHEN duration.between(e1.timestamp , e2.timestamp).minutes < 3 AND (e1) - [:DF_EMP] - (e2) 
        THEN e1.batchID
    else 
        e2.batchID
            END'''
session.run(query)

In [48]:
query=f'''MATCH (e:Event) UNWIND e.batchID AS id_val
WITH DISTINCT id_val
MERGE (:Batch {{ID:id_val}})'''
session.run(query)

In [49]:
query=f'''MATCH (e:Event) 
MATCH (b:Batch) 
WHERE e.batchID = b.ID
merge (e)-[:BATCHED]->(b)'''
session.run(query)

In [50]:
query = f'''MATCH (e:Event)-[:BATCHED]->(b:Batch)
WITH e.timestamp AS timestamp, b
with MIN(timestamp) AS minTimestamp, MAX(timestamp) AS maxTimestamp, b
set b.startTimestamp = minTimestamp, b.endTimestamp = maxTimestamp'''
session.run(query)

In [51]:
query=f'''MATCH (e: Event)
where e.Activity in ['Composición de cargas', 'Carga de esterilizador liberada']
WITH e AS nodes ORDER BY e.timestamp, ID(e)
WITH collect(nodes) AS event_node_list
UNWIND range(0, size(event_node_list)-2) AS i
WITH event_node_list[i] AS e1, event_node_list[i+1] AS e2
where e1.batchID = e2.batchID and e1.additionalInfo1 is null and e2.additionalInfo1 is not null
set e1.additionalInfo1 = e2.additionalInfo1'''
session.run(query).single()

## Create Containers

In [52]:
query = f'''match (k:Kit) - [:HAS_UNIT] -> (e:Entity)
where e.ID contains 'CONT'
set k.isWashingMachineContainer = True
'''
session.run(query).single()

In [53]:
query = f'''match (k:Kit {{isWashingMachineContainer: True}}) - [:HAS_UNIT] -> (e:Entity) <- [:HAS_RUN] - (r:Run)
where (r.activities contains "Composición de cargas" or r.activities contains "Carga de esterilizador liberada" or r.activities contains "Comisionado")
set k.isWashingMachineContainer = False'''
session.run(query).single()

In [54]:
query = f'''match (k:Kit {{isWashingMachineContainer: True}}) - [:HAS_UNIT] -> (e:Entity) <- [:CORR] - (ev:Event)
where (ev.Activity = "Composición de cargas" or ev.Activity = "Carga de esterilizador liberada" or ev.Activity = "Comisionado")
set k.isWashingMachineContainer = False'''
session.run(query).single()

## Create Contained Kits

In [55]:
query = f'''match (e:NewKitEvent {{Activity: "Montaje"}})
WITH collect(distinct e.KitID) as kids 
UNWIND range(0, size(kids)-1) AS i
with kids[i] as kid
match (k:Kit) - [:HAS_UNIT] -> (e:Entity {{ID: kid}})
set k.isContainedKit = True'''
session.run(query).single()

In [56]:
query = f'''match (k:Kit {{isContainedKit: True}}) - [:HAS_UNIT] -> (e:Entity) - [:HAS_RUN] -> (r:Run)
where (r.activities contains 'Entrada Material Sucio' 
or r.activities contains 'Cargado en carro L+D' 
or r.activities contains 'Carga L+D iniciada' 
or r.activities contains 'Carga L+D liberada')
set k.isContainedKit = False'''
session.run(query).single()

In [57]:
query = f'''match (k:Kit {{isContainedKit: True}}) - [:HAS_UNIT] -> (e:Entity) <- [:CORR] - (ev:Event)
where (ev.Activity = 'Entrada Material Sucio' 
or ev.Activity = 'Cargado en carro L+D' 
or ev.Activity = 'Carga L+D iniciada'
or ev.Activity = 'Carga L+D liberada')
set k.isContainedKit = False'''
session.run(query).single()